# PydanticAI example with LiteLLM Proxy and SAP LLMs

## [How PydanticAI works](https://ai.pydantic.dev/agents/)
## [LiteLLM Proxy setup for SAP models](../proxy_set_up.ipynb)

## Installation

In [ ]:
%pip install pydantic-ai litellm

## Set env variables
Add the following variables from the service key in a file called ".env" and put it in the same folder where you run the notebook:
```
LITELLM_PROXY_API_KEY=sk-1234
PROXY_BASE_URL=http://localhost:4000
```

## Run the PydanticAI with LiteLLM and SAP LLMs

In [2]:
import os

import litellm
from dotenv import load_dotenv
from pydantic_ai import Agent, RunContext
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.litellm import LiteLLMProvider

Load your credentials as environment variables.

In [3]:
litellm.use_litellm_proxy = True
load_dotenv()
api_base = os.getenv("PROXY_BASE_URL")
api_key = os.getenv("LITELLM_PROXY_API_KEY")

Define the model with OpenAIChatModel object with LiteLLMProvider object with your proxy credentials

In [4]:
model = OpenAIChatModel(
    "sap/gpt-5",
    provider=LiteLLMProvider(
        api_base=api_base,
        api_key=api_key,
    ),
)

Define the agent

In [5]:
agent = Agent(
    model=model,
    system_prompt="You are a helpful weather assistant. "
    "When the user send you asks a specific city, "
    "use the 'get_weather' tool to find the information about the weather. "
    "Aser with TV weather report in two sentences, include small jok",
)

Define the agent tool.

In [6]:
@agent.tool
def get_weather(city: RunContext[str]) -> str:
    """Mock function"""
    city_normalized = city.prompt.lower().replace(" ", "")

    mock_weather_db = {
        "newyork": "The weather in New York is sunny with a temperature of 25°C.",
        "london": "It's cloudy in London with a temperature of 15°C.",
        "tokyo": "Tokyo is experiencing light rain and a temperature of 18°C.",
    }

    if city_normalized in mock_weather_db:
        return mock_weather_db[city_normalized]
    else:
        return f"The weather in {city} is sunny with a temperature of 20°C."

Run agent

In [9]:
result = await agent.run("London")
print(result.output)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/typing.py:904: RuntimeWarning: coroutine 'AbstractAgent.run' was never awaited
  code = compile(arg_to_compile, '<string>', 'eval')


Good evening, London! We’ve got cloudy skies hovering over the city with a cool 15°C—keep that light jacket handy. On the bright side, the sun’s just playing hide-and-seek; unfortunately, it’s really good at hiding today!
